In [3]:
import json
from collections import Counter
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 100
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import sys
import os
import argparse
from os.path import dirname, realpath

import hashlib
import datetime
import datetime as datetime
import sklearn
import statistics
sys.path.append("./src/")
#from disrisknet.utils.parsing import parse_args
import pdb

src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()

from sklearn.metrics import confusion_matrix

In [4]:
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/allChars_DF.csv'))
from sklearn.metrics import confusion_matrix


In [5]:
logpath = os.path.join(src_path ,'logs_transformer_vte/F/', "0d071737ad78312a4c72357d28bd336b.results.test_preds")


In [6]:
 
with open(logpath, 'rb') as f:
    R6 = pickle.load(f)
    p6 = np.array(R6['probs'])

pat_ids = (testDF['patient_id'] ).astype(int)
 

In [9]:
R6.keys()

dict_keys(['golds', 'probs', 'patient_golds', 'exams', 'pids', 'dates', 'censor_times', 'days_to_final_censors'])

In [10]:
testDF['pids'] = pat_ids.apply(md5)

g6 = pd.DataFrame({'p6': R6['probs'] ,    
                   'pids':np.array(R6['pids']),    
                   'model_date':np.array(R6['dates'])
                  })

M = pd.merge(g6, testDF)

m = M[["patient_id","dx_date", "index_date", "model_date",  "outcome","outcome_date","p6" ]]
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])

m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)


In [11]:
R6.keys()

dict_keys(['golds', 'probs', 'patient_golds', 'exams', 'pids', 'dates', 'censor_times', 'days_to_final_censors'])

In [12]:
M.keys()

Index(['p6', 'pids', 'model_date', 'Unnamed: 0', 'patient_id', 'outcome',
       'dob', 'outcome_date', 'obs_time_end', 'index_date', 'dx_date',
       'obs_time_start', 'ev_length', 'len0', 't0', 'len30', 't30', 'len90',
       't90', 'len180', 't180', 'ks', 'sex', 'bmi', 'race'],
      dtype='object')

In [13]:
M

,p6,pids,model_date,Unnamed: 0,patient_id,outcome,dob,outcome_date,obs_time_end,index_date,...,len30,t30,len90,t90,len180,t180,ks,sex,bmi,race
0,[0.55457234],800dd2cea2c86677c65656fbf7796c6a,2008-01-07T00:00:00,51341,1004168442,False,1927-09-14,2008-01-08,2008-01-08,2007-10-01,...,1368,3017 days,1368,3017 days,1368,3017 days,2,0,1,1
1,[0.5106311],5e17ad39df9e4758bb82f3751f747a66,2013-10-02T00:00:00,57799,1012231226,True,1944-03-10,2014-04-06,2015-08-28,2011-09-21,...,2120,3141 days,2075,3085 days,1983,2984 days,5,0,2,1
2,[0.51725596],bf451aea21dc287204548367e0816189,2013-11-22T00:00:00,53745,1012370247,True,1943-10-12,2015-04-24,2016-01-25,2011-11-21,...,2938,3505 days,2861,3436 days,2664,3349 days,4,0,3,5
3,[0.30241328],835159575e9bffd30ab38cca7b1cfef9,2022-05-12T00:00:00,53003,1011593915,False,1949-09-24,2023-04-28,2023-04-28,2020-05-13,...,536,5455 days,536,5455 days,536,5455 days,1,0,3,5
4,[0.1670009],259e4fd93d73517a43b68d3764fb0d5b,2006-05-27T00:00:00,54854,1002258530,False,1935-09-30,2006-07-04,2006-07-04,2006-04-05,...,258,2318 days,258,2318 days,258,2318 days,4,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,[0.09159892],8632c8bf233041a0de7843144d771f1c,2016-08-18T00:00:00,57089,1002873149,False,1952-09-03,2022-11-16,2022-11-16,2014-08-13,...,396,4482 days,396,4482 days,396,4482 days,2,1,1,1
1995,[0.07377031],ec856314270a319ec6c175276fa01b9a,2021-01-27T00:00:00,53710,1004353036,False,1946-04-29,2023-05-30,2023-05-30,2019-01-11,...,356,7300 days,356,7300 days,356,7300 days,1,0,4,1
1996,[0.47808862],8badb8fecd272b670a6f5dd611ae2401,2012-08-31T00:00:00,51541,1001719180,False,1946-08-22,2012-12-27,2012-12-27,2011-04-01,...,844,4299 days,844,4299 days,844,4299 days,2,0,2,1
1997,[0.7329938],7da4fcbec04b14db6c3cc6273c003560,2019-09-13T00:00:00,58496,1016579496,False,1954-07-19,2023-05-23,2023-05-23,2017-09-15,...,2145,3162 days,2145,3162 days,2145,3162 days,3,0,0,0


In [14]:
sklearn.metrics.roc_auc_score(m1['outcome'],  m1['p6'], average = 'samples')


0.6900941656032465

In [15]:
( confusion_matrix (m1['outcome'],  m1['p6']>.5) )

array([[1445,  372],
       [ 104,   78]], dtype=int64)

In [76]:
# how many people had event within 6 months of index date

In [77]:
m1.to_csv('output/CC8_test180.csv') 

In [8]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 

In [27]:
statistics.quantiles( M['len180'],n=20)

[289.0,
 390.0,
 471.0,
 541.0,
 610.0,
 677.0,
 743.0,
 809.0,
 879.0,
 952.0,
 1036.0,
 1128.0,
 1230.0,
 1343.0,
 1482.5,
 1647.0,
 1880.9,
 2189.0,
 2772.0]

In [23]:
plt.hist(M['len180'])

(array([2.2715e+04, 1.8050e+03, 1.3000e+02, 2.5000e+01, 1.2000e+01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.0000e+00]),
 array([1.00000e+00, 2.37810e+03, 4.75520e+03, 7.13230e+03, 9.50940e+03,
        1.18865e+04, 1.42636e+04, 1.66407e+04, 1.90178e+04, 2.13949e+04,
        2.37720e+04]),
 <BarContainer object of 10 artists>)